# Day 3

In [ ]:
import pykx as kx

🩺 After days of exposure to this unknown environment, you begin to notice changes in your physical state. Luckily, a wearable vitals monitor survived the crash. It has recorded sporadic biometric readings like heart rate and hydration levels.

To understand the impact of your environment on your health, you decide to align your vitals data with the weather conditions recorded in Day 1 — and visualize how your body reacted.

📂 Data
You now have two datasets:

weather (from day 1)

| time                    | sensorId | temperatureF | humidity |
|-------------------------|----------|--------------|----------|
| 2026-01-25 00:00:00.000 | 67       | 74.49463     | 62.2822  |
| 2026-01-25 00:00:00.500 | 91       | 73.88359     | 62.53392 |
| 2026-01-25 00:00:01.000 | 42       | 72.1795      | 62.53017 |
| 2026-01-25 00:00:01.500 | 84       | 72.90408     | 64.90296 |
| ...                     |          |              |          |

vitals.csv with heart rate and hydration readings every few seconds to few minutes

| time                   | heartRate | hydration |
|------------------------|-----------|-----------|
| 2026.01.25D00:01:47.00 | 50        | 59.68     |
| 2026.01.25D00:06:22.00 | 55        | 66.99     |
| 2026.01.25D00:08:13.00 | 58        | 67.44     |
| 2026.01.25D00:09:29.00 | 55        | 63.28     |
| ...                    |           |           |

🧩 Challenge

Into a q instance load:
- weather data from day 1
- vitals.csv into a table called vitals with data types:
    - time as timestamp
    - heartRate as integer
    - hydration as real

- For each vitals reading, find the closest previous weather observation.
- Connecting to your q instance using Python, or using another KDB-X integration, plot a scatter plot of heart rate vs temperature.
- Using your visualization or otherwise, identify periods where heart rate exceeds 100 bpm and temperature is above 77°F.

In [ ]:
vitals: `time xasc ("PIE";1#",")0:`:vitals.csv;

.pq:use`kx.pq;
t:.pq.pq`:weather.parquet;
weather: `time xasc select from t;

closest: aj[`time; vitals; weather];
\l pykx.q

In [ ]:
ans1: first[0!desc select count i by sensorId from closest]`sensorId;
ans2: count select from closest where heartRate > 100, temperatureF>77f;
ans3: first[0!desc select count i by hour:60 xbar time.minute from select from closest where heartRate > 100, temperatureF>77f]`hour;
show (ans1; ans2; ans3);

In [ ]:
%%py
import matplotlib.pyplot as plt
closest = kx.q("closest").pd()
closest['condition'] = (closest.heartRate>100) & (closest.temperatureF>77)
colors = closest['condition'].map({True: 'red', False: 'blue'})

_, ax = plt.subplots(1, 1)
closest.plot(
    x='heartRate', 
    y='temperatureF', 
    kind='scatter', 
    c=colors,
    ax=ax, 
    alpha=0.5, 
    title='Heart Rate vs. Ambient Temperature'
)

plt.show()